# Introduction

 In this project, i am going to build a K-Means model to create clusters of respondents to the Survey of Consumer Finances.  I will examine all the features, selecting five to create clusters with. After building the model and choosing an appropriate number of clusters, i will create visualizations for multi-dimensional clusters in a 2D scatter plot using  principal component analysis (PCA).



In [26]:
# Import libraries here
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Prepare Data



First, i need to import the file, scfp2019excel.zip. I will use the pandas read_csv () method to import and decompress the data file.

In [2]:
df = pd.read_csv("/home/tatenda/Desktop/ds_projects/scfp2019excel.zip")
print("df shape:", df.shape)
df.head()

df shape: (28885, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
0,1,11,6119.779308,2,75,6,12,4,2,0,...,5,3,6,3,2,10,6,6,3,3
1,1,12,4712.374912,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2
2,1,13,5145.224455,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2
3,1,14,5297.663412,2,75,6,12,4,2,0,...,5,2,6,2,1,10,4,4,2,2
4,1,15,4761.812371,2,75,6,12,4,2,0,...,5,3,6,3,1,10,5,5,2,2


Below is a function function that returns a DataFrame of households whose net worth is less than $2 million and that have been turned down for credit or feared being denied credit in the past 5 years ( "TURNFEAR" and "NETWORTH" columns). According to the data dictionary, TURNFEAR==1 implies households that have been turned down or feared being turned down for credit.

In [3]:
def wrangle(filepath):
    #Read file into DataFrame
    df = pd.read_csv(filepath)
    mask = (df["TURNFEAR"]==1) & (df["NETWORTH"]<2e6)
    df = df[mask]
    return df

In [4]:
df = wrangle("/home/tatenda/Desktop/ds_projects/scfp2019excel.zip")
print(df.shape)
df.head()

(4418, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
5,2,21,3790.476607,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
6,2,22,3798.868505,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,3,2,2
7,2,23,3799.468393,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
8,2,24,3788.076005,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
9,2,25,3793.066589,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2


## Explore

 We want to make clusters using several features, but which of the 351 features should we choose? Often times, this decision will be made for you. For example, a stakeholder could give you a list of the features that are most important to them. If you don't have that limitation, though, another way to choose the best features for clustering is to determine which numerical features have the largest variance. That's what i will do here.


 Now we calculate the variance for all the features in df, and create a Series top_ten_var with the 10 features with the largest variance.

In [5]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)
top_ten_var

PLOAN1      1.140894e+10
ACTBUS      1.251892e+10
BUS         1.256643e+10
KGTOTAL     1.346475e+10
DEBT        1.848252e+10
NHNFIN      2.254163e+10
HOUSES      2.388459e+10
NETWORTH    4.847029e+10
NFIN        5.713939e+10
ASSET       8.303967e+10
dtype: float64

It is better and easier to see the above information in the form of a visualisation. So i will use plotly express to create a horizontal bar chart of top_ten_var.

In [27]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x=top_ten_var,
    y=top_ten_var.index,
    title="SCF: High Variance Features"
)
fig.update_layout(xaxis_title="Variance", yaxis_title="Feature")
fig.show()
plt.savefig("images1.png", dpi=150)

<Figure size 432x288 with 0 Axes>

One thing that we are seeing in  this project is that many of the wealth indicators are highly skewed, with a few outlier households having enormous wealth. Those outliers can affect our measure of variance. Let's see if that's the case with one of the features from top_five_var

I will use plotly express to create a horizontal boxplot of "NHNFIN" to determine if the values are skewed

In [28]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame=df,
    x="NHNFIN",
    title= "Distribution of Non-home, Non-Financial Assets"
)
fig.update_layout(xaxis_title="Value [$]")

fig.show()
plt.savefig("images2.png", dpi=150)

<Figure size 432x288 with 0 Axes>

The dataset is massively right-skewed because of the huge outliers on the right side of the distribution. Even though we already excluded households with a high net worth with our `wrangle` function, the variance is still being distorted by some extreme outliers.

The best way to deal with this is to look at the **trimmed variance**, where we remove extreme values before calculating variance. We can do this using the `trimmed_variance` function from the `SciPy` library.

Let's calculate the trimmed variance for the features in df. The calculation excludes the top and bottom 10% of observations. Then we will create a Series top_ten_trim_var with the 10 features with the largest variance.

In [8]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var).sort_values().tail(10)

top_ten_trim_var

WAGEINC     5.550737e+08
HOMEEQ      7.338377e+08
NH_MORT     1.333125e+09
MRTHEL      1.380468e+09
PLOAN1      1.441968e+09
DEBT        3.089865e+09
NETWORTH    3.099929e+09
HOUSES      4.978660e+09
NFIN        8.456442e+09
ASSET       1.175370e+10
dtype: float64

Let's use plotly express to create a horizontal bar chart of top_ten_trim_var

In [29]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x=top_ten_trim_var,
    y=top_ten_trim_var.index,
    title="SCF: High Variance Features"
)

fig.update_layout(xaxis_title= "Trimmed Variance", yaxis_title ="Feature")

fig.show()
plt.savefig("images3.png", dpi=150)

<Figure size 432x288 with 0 Axes>

There are three things to notice in this plot. First, the variances have decreased a lot. In our previous chart, the x-axis went up to \\$80 billion; this one goes up to \\$12 billion. Second, the top 10 features have changed a bit. All the features relating to business ownership (`"...BUS"`) are gone. Finally, we can see that there are big differences in variance from feature to feature. For example, the variance for `"WAGEINC"` is around than \\$500 million, while the variance for `"ASSET"` is nearly \\$12 billion. In other words, these features have completely different scales. This is something that we'll need to address before we can make good clusters.

Let's generate a list high_var_cols with the column names of the five features with the highest trimmed variance.

In [10]:
high_var_cols = top_ten_trim_var.tail(5).index.to_list()
high_var_cols

['DEBT', 'NETWORTH', 'HOUSES', 'NFIN', 'ASSET']

## Split
Let's create the feature matrix X. It should contain the five columns in high_var_cols

In [11]:
X = df[high_var_cols]
print("X shape:", X.shape)
X.head()

X shape: (4418, 5)


,DEBT,NETWORTH,HOUSES,NFIN,ASSET
5,12200.0,-6710.0,0.0,3900.0,5490.0
6,12600.0,-4710.0,0.0,6300.0,7890.0
7,15300.0,-8115.0,0.0,5600.0,7185.0
8,14100.0,-2510.0,0.0,10000.0,11590.0
9,15400.0,-5715.0,0.0,8100.0,9685.0


# Build Model

## Iterate

During the EDA, we saw that we had a scale issue among our features. That issue can make it harder to cluster the data, so we'll need to fix that to help our analysis along. One strategy we can use is **standardization**, a statistical method for putting all the variables in a dataset on the same scale. Let's explore how that works here. Later, we'll incorporate it into our model pipeline.

 Create a StandardScaler transformer, use it to transform the data in X, and then put the transformed data into a DataFrame named X_scaled.

In [12]:
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)  

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data,columns=X.columns)

print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

X_scaled shape: (4418, 5)


,DEBT,NETWORTH,HOUSES,NFIN,ASSET
0,-0.445075,-0.377486,-0.48231,-0.474583,-0.498377
1,-0.442132,-0.368401,-0.48231,-0.464541,-0.490047
2,-0.422270,-0.383868,-0.48231,-0.467470,-0.492494
3,-0.431097,-0.358407,-0.48231,-0.449061,-0.477206
4,-0.421534,-0.372966,-0.48231,-0.457010,-0.483818


Let's use a for loop to build and train a K-Means model where n_clusters ranges from 2 to 12 (inclusive). The model includes a StandardScaler. Each time a model is trained, calculate the inertia and add it to the list inertia_errors, then calculate the silhouette score and add it to the list silhouette_scores.

In [13]:
n_clusters = range(2, 12)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    #Build model
    model = make_pipeline(StandardScaler(), KMeans(n_clusters=k, random_state=42))
    #Train model
    model.fit(X)
    #Calculate inertia
    inertia_errors.append(model.named_steps["kmeans"].inertia_)
    #Calculate silhoutte score
    silhouette_scores.append(
        silhouette_score(X,model.named_steps["kmeans"].labels_)
    )

    
print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Inertia: [11028.058082607142, 7190.526303575357, 5921.636429837139]

Silhouette Scores: [0.7464502937083215, 0.7044601307791996, 0.6920745639331739]


Let's use plotly express to create a line plot that shows the values of inertia_errors as a function of n_clusters. This is the elbow method that will help us to determine the number of clusters to use.

In [30]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=inertia_errors, title="K-Means Model: Inertia vs Number of Clusters"
)
fig.update_layout(xaxis_title="Number of Clusters (k)", yaxis_title="Inertia")
fig.show()
plt.savefig("images4.png", dpi=150)

<Figure size 432x288 with 0 Axes>

You can see that the line starts to flatten out around 4 or 5 clusters.
Let's make another line plot based on the silhouette scores.


Let's use plotly express to create a line plot that shows the values of silhouette_scores as a function of n_clusters

In [31]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters,
    y=silhouette_scores,
    title="K-Means Model: Silhouette Score vs Number of Clusters"
)
fig.update_layout(
    xaxis_title="Number of Clusters", yaxis_title="Silhouette Score"
)

fig.show()
plt.savefig("images5.png", dpi=150)

<Figure size 432x288 with 0 Axes>

 We can see that the best silhouette scores occur when there are 3 or 4 clusters. 

Putting the information from this plot together with our inertia plot, it seems like the best setting for `n_clusters` will be 4. 

Let's build and train a new k-means model named final_model,using the information we gained from the two plots above to set an appropriate value for the n_clusters argument.

In [16]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

# Communicate

In [17]:
#Extraxting labels created by final_model during training
labels = final_model.named_steps["kmeans"].labels_
print(labels[:5])

[0 0 0 0 0]


In [18]:
# DataFrame xgb that contains the mean values of the features in X for each of the clusters in final_model
xgb = X.groupby(labels).mean()
xgb

,DEBT,NETWORTH,HOUSES,NFIN,ASSET
0,25475.184486,13336.327716,12657.640906,2.604130e+04,3.881151e+04
1,126752.280000,900314.904000,289648.000000,7.349837e+05,1.027067e+06
2,215657.219388,161855.510204,248186.862245,3.194630e+05,3.775127e+05
3,725213.134328,778260.298507,819776.119403,1.289561e+06,1.503473e+06


Let's use plotly express to create a side-by-side bar chart from xgb that shows the mean of the features in X for each of the clusters in  final_model

In [32]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title="Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")
fig.show()
plt.savefig("images6.png", dpi=150)

<Figure size 432x288 with 0 Axes>

Remember that our clusters are based partially on `NETWORTH`, which means that the households in the 0 cluster have the smallest net worth, and the households in the 2 cluster have the highest. Based on that, there are some interesting things to unpack here.

First, let's look at the `DEBT` variable. One might think that it would scale as net worth increases, but it doesn't. The lowest amount of debt is carried by the households in cluster 2, even though the value of their houses (shown in green) is roughly the same. You can't *really* tell from this data what's going on, but one possibility might be that the people in cluster 2 have enough money to pay down their debts, but not quite enough money to leverage what they have into additional debts. The people in cluster 3, by contrast, might not need to worry about carrying debt because their net worth is so high. 

Finally, since we started out this project looking at home values, take a look at the relationship between `DEBT` and `HOUSES`. The value of the debt for the people in cluster 0 is higher than the value of their houses, suggesting that most of the debt being carried by those people is tied up in their mortgages — if they own a home at all. Contrast that with the other three clusters: the value of everyone else's debt is lower than the value of their homes.

Let's create a PCA transformer, use it to reduce the dimensionality of the data in X to 2, and then put the transformed data into a DataFrame named X_pca. The columns of X_pca should be named "PC1" and "PC2"

In [20]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns=["PC1","PC2"])

print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca shape: (4418, 2)


,PC1,PC2
0,-221525.424530,-22052.273003
1,-217775.100722,-22851.358068
2,-219519.642175,-19023.646333
3,-212195.720367,-22957.107039
4,-215540.507551,-20259.749306


Let's use plotly express to create a scatter plot of X_pca using seaborn. Be sure to color the data points using the labels generated by your final_model

In [33]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x="PC1",
    y="PC2",
    color=labels.astype(str),
    title="PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()
plt.savefig("images7.png", dpi=150)

<Figure size 432x288 with 0 Axes>